In [26]:
!pip install -e ..

Obtaining file:///C:/Users/lenovo/Documents/GitHub/Project
  Attempting uninstall: PDS-Project
    Found existing installation: PDS-Project 0.0.1.dev1
    Uninstalling PDS-Project-0.0.1.dev1:
      Successfully uninstalled PDS-Project-0.0.1.dev1
  Running setup.py develop for PDS-Project


In [ ]:
#Imports
import os
import pandas as pd

#Get Data
print("Read data...")
path = os.path.join(os.path.join(os.getcwd(), "../nextbike/data"), "input/nuremberg.csv")
df = pd.read_csv(path)


#Prepare Columns
print("Prepare columns...")
df_clean = df.drop(["Unnamed: 0", 
                    "p_spot", 
                    "p_place_type", 
                    "p_bike", 
                    "b_bike_type", 
                    "p_bikes", 
                    "p_uid", 
                    "p_number"], axis=1)
df_clean.rename({"p_lat" : "Latitude", 
                 "p_lng":"Longitude", 
                 "p_name":"Place",
                 "b_number":"Bike Number"}, axis=1, inplace=True)


#Drop Duplicates
print("Drop duplicates...")
df_clean_unique=df_clean.drop_duplicates(subset=df_clean.columns.difference(["Longitude", "Latitude"]))


#Drop trip first/last
print("Filter on start/end...")
df_clean_unique_trip = df_clean_unique[(df_clean_unique["trip"]=="start")|(df_clean_unique["trip"]=="end")]

#Eliminate Noise
print("Eliminate Noise")
sr_noise = (df_clean_unique_trip['trip'] != df_clean_unique_trip['trip'].shift())
df_clean_unique_trip['Noisy Entry']=sr_noise
df_final = df_clean_unique_trip[df_clean_unique_trip["Noisy Entry"]==True]

#split, reindex, merge
print("Create Master DataFrame")
df_s = df_final[df_final["trip"]=="start"]
df_e = df_final[df_final["trip"]=="end"]
df_e["index"]=range(0, len(df_e))
df_s["index"]=range(0, len(df_s))
df_merged=df_s.merge(df_e, left_on=df_s["index"], right_on=df_e["index"], suffixes=('_start', '_end'))
df_merged.drop(["key_0", 
                "trip_start", 
                "Noisy Entry_start", 
                "index_start", 
                "Bike Number_end", 
                "trip_end", 
                "Noisy Entry_end", 
                "index_end"], axis=1, inplace = True, errors="ignore")
df_merged.rename({"datetime_start":"Start Time", "Bike Number_start":"Bike Number", "datetime_end":"End Time"}, axis=1)

#Save final df
print("Save csv")
df_merged.to_csv("../nextbike/data/output/final.csv")

print("DONE")
print(df_merged.head())

Read data...
Prepare columns...
Drop duplicates...
Filter on start/end...
Eliminate Noise


C:\Users\lenovo\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Create Master DataFrame


C:\Users\lenovo\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lenovo\miniconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Save csv
